In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np

In [3]:
from visualizer import Visualizer

In [4]:
from cpp_sisso import get_max_number_feats, get_estimate_n_feat_next_rung, generate_fs, SISSOClassifier, generate_phi_0_from_csv, FeatureSpace

In [5]:
df_train = pd.read_pickle('./data/training_set')

In [6]:
# dataset for SISSO - tetradymites tutorial
phi_0, prop_unit, prop, prop_test, task_sizes_train, task_sizes_test, leave_out_inds = generate_phi_0_from_csv(
df_train, "Class", cols='all', task_key=None, leave_out_frac=0.0
)
feat_space = generate_fs(
    phi_0, 
    prop, 
    task_sizes_train, 
    ["add", "sub", "mult", "div", "abs_diff", "sq", "cb", "sqrt", "cbrt", "inv", "abs"],  
    "classification",
    0, 
    20
)
sisso = SISSOClassifier(
    feat_space,
    prop_unit,
    prop,
    prop_test,
    task_sizes_train,
    task_sizes_test,
    leave_out_inds,
    2,
    10,
    10
)
sisso.fit()
model = sisso.models[1][0]
classified=model.prop_train
compounds = df_train.index.to_list()
df=pd.DataFrame(data={
    "Compound":compounds,
    "Classification":classified})
for feat in sisso.models[sisso.n_dim-1][0].feats:
    df[str(feat)]=feat.value
classes = ['Topological insulators', 'Trivial insulators']


In [7]:
features = []
for feat in sisso.models[sisso.n_dim-1][0].feats:
    features.append(feat)

In [8]:
df['Classes']=np.where(df['Classification']==0,classes[0],classes[1])

In [9]:
df=df.set_index('Compound')

In [10]:
df

,Classification,abs[[[X16C+X16D]*[Z11E/X16B]]-[[Z11C+Z11D]+abs[Z11C-Z11D]]],[[[Z11A+Z11B]*[Z11C+Z11D]]-abs[[Z11A*Z11D]-[Z11B*Z11C]]],Classes
Compound,,,,
Sb_Sb_Te_Te_Te,0.0,3.5512,10608.0,Topological insulators
As_Bi_Te_Te_S,0.0,70.4160,9464.0,Topological insulators
Bi_Bi_Te_Se_Se,0.0,25.3960,12782.0,Topological insulators
Bi_Sb_Te_S_Te,0.0,15.2200,7788.0,Topological insulators
Bi_As_Te_Se_Te,0.0,7.3945,8870.0,Topological insulators
...,...,...,...,...
Bi_Sb_S_S_Se,1.0,53.5800,3776.0,Trivial insulators
Sb_Sb_Te_Se_S,1.0,67.5510,7854.0,Trivial insulators
Sb_Sb_Se_Te_S,1.0,67.5510,7854.0,Trivial insulators


In [11]:
visualizer=Visualizer(df, sisso, classes, features, ['Classification'])

In [12]:
visualizer.show()